In [5]:
import requests
import json
import time
import pandas as pd
import numpy as np
from WebCrawlerLibrary import gen_query_url, parse_response, extract_data_from_api, query_start_date
from datetime import datetime, timedelta

In [2]:
# Design dataframe
API_COLUMNS = ["Title", "Description", "NewsURL", "PublishedTime", "SourceName" , "SourceURL"]

api_content = pd.read_csv("../Data/NewsContent.csv", index_col=[0])
# api_content = pd.read_csv("../Data/NewsContent.csv")

In [3]:
api_content_current_session = pd.DataFrame(columns=API_COLUMNS)

In [14]:
datetime.now().strftime(datetime_format)

'2020-07-30'

In [6]:
# Process query days pipeline
datetime_format = "%Y-%m-%d"
# start_date = datetime(2020,7,20)
start_date = query_start_date(api_content)
query_index_date = start_date 

status_code = 200
counter = 0
max_count = 49
first_time = False


while counter < max_count or status_code == 200 and query_index_date != datetime.now().strftime(datetime_format):
    # counter does not surpass 100 queryies per day
    # status_code is still successful
    # index date has yet to surpass today
    
    # Generate URLs
    query_index_date_next = query_index_date + timedelta(days=1)
    url = gen_query_url(query_index_date.strftime(datetime_format), query_index_date_next.strftime(datetime_format) )
    
    # Parse URLs
    response = requests.get(url)
    
    # get Status_code and content
    status_code, output_content = parse_response(response)
    
    # Parse content
    if status_code == 200 and output_content != "no returns":    
        new_api_data = extract_data_from_api(output_content)
    
    if counter == 0 and first_time == True:
        api_content = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(api_content)
    else :
        new_api_df = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(new_api_df)
        api_content = api_content.append(new_api_df, ignore_index = True).drop_duplicates()
        api_content_current_session = api_content_current_session.append(new_api_df, ignore_index = True).drop_duplicates()
    

    # Post Processing conditions
    query_index_date = query_index_date_next
    counter += 1
    if counter == max_count:
        print("Has Reached Max Count of " + str(counter))
    if status_code != 200:
        print("Status Code " + str(status_code))
    time.sleep(5)

https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-07-29&maxdate=2020-07-30&in=title
https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-07-30&maxdate=2020-07-31&in=title
https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-07-31&maxdate=2020-08-01&in=title


KeyboardInterrupt: 

In [7]:
api_content

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,TSA issues new directive for anyone entering t...,issued a new directive effective Sunday requir...,https://www.msn.com/en-us/news/us/tsa-issues-n...,2020-02-01,CNN on MSN.com,https://www.msn.com
1,"As coronavirus spreads, TSA issues new directi...",issued a new directive effective Sunday requir...,https://edition.cnn.com/2020/02/02/politics/ts...,2020-02-01,CNN,https://edition.cnn.com
2,Neenah family secures seats on flight from cor...,"NEENAH — Daisy, Abigail, and Adalynn Roth have...",https://fox6now.com/2020/02/02/neenah-family-s...,2020-02-01,FOX6 Milwaukee,https://fox6now.com
3,"US companies suspend China operations, restric...","Disney, Tesla, airlines and other companies wi...",https://www.cnbc.com/2020/02/01/coronavirus-co...,2020-02-01,CNBC,https://www.cnbc.com
4,Coronavirus: US confirms eighth case with quar...,Federal authorities say Massachusetts patient ...,https://www.theguardian.com/world/2020/feb/01/...,2020-02-01,The Guardian,https://www.theguardian.com
...,...,...,...,...,...,...
1496,US GDP plunged by a record 33% annual rate in ...,A leading-edge research firm focused on digita...,https://www.businessinsider.com/us-q2-gdp-reco...,2020-07-30 09:26:00 UTC,Business Insider,https://www.businessinsider.com
1497,Melbourne struggles with second coronavirus sp...,COVID-19 is considered a novel coronavirus bec...,https://www.pri.org/stories/2020-07-30/melbour...,2020-07-30 09:13:00 UTC,PRI,https://www.pri.org
1498,Misinformation on coronavirus proves highly co...,“It is a real challenge in terms of trying to ...,https://www.ksl.com/article/46782225/misinform...,2020-07-30 07:52:00 UTC,KSL,https://www.ksl.com
1499,Bill Gates: We will have a coronavirus vaccine...,We talked to Bill Gates about the coronavirus ...,https://www.businessinsider.com/bill-gates-cor...,2020-07-30 07:48:00 UTC,Business Insider,https://www.businessinsider.com


In [8]:
# temp = api_content_current_session.copy()
# temp = temp.drop_duplicates(["Title" , "PublishedTime"])
# api_content = api_content.append(temp, ignore_index = True).drop_duplicates()


In [11]:
api_content.PublishedTime   = pd.to_datetime(api_content.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')
api_content_current_session.PublishedTime   = pd.to_datetime(api_content_current_session.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')

api_content = api_content.drop_duplicates(["Title", "PublishedTime"])
api_content

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,TSA issues new directive for anyone entering t...,issued a new directive effective Sunday requir...,https://www.msn.com/en-us/news/us/tsa-issues-n...,2020-02-01,CNN on MSN.com,https://www.msn.com
1,"As coronavirus spreads, TSA issues new directi...",issued a new directive effective Sunday requir...,https://edition.cnn.com/2020/02/02/politics/ts...,2020-02-01,CNN,https://edition.cnn.com
2,Neenah family secures seats on flight from cor...,"NEENAH — Daisy, Abigail, and Adalynn Roth have...",https://fox6now.com/2020/02/02/neenah-family-s...,2020-02-01,FOX6 Milwaukee,https://fox6now.com
3,"US companies suspend China operations, restric...","Disney, Tesla, airlines and other companies wi...",https://www.cnbc.com/2020/02/01/coronavirus-co...,2020-02-01,CNBC,https://www.cnbc.com
4,Coronavirus: US confirms eighth case with quar...,Federal authorities say Massachusetts patient ...,https://www.theguardian.com/world/2020/feb/01/...,2020-02-01,The Guardian,https://www.theguardian.com
...,...,...,...,...,...,...
1496,US GDP plunged by a record 33% annual rate in ...,A leading-edge research firm focused on digita...,https://www.businessinsider.com/us-q2-gdp-reco...,2020-07-30,Business Insider,https://www.businessinsider.com
1497,Melbourne struggles with second coronavirus sp...,COVID-19 is considered a novel coronavirus bec...,https://www.pri.org/stories/2020-07-30/melbour...,2020-07-30,PRI,https://www.pri.org
1498,Misinformation on coronavirus proves highly co...,“It is a real challenge in terms of trying to ...,https://www.ksl.com/article/46782225/misinform...,2020-07-30,KSL,https://www.ksl.com
1499,Bill Gates: We will have a coronavirus vaccine...,We talked to Bill Gates about the coronavirus ...,https://www.businessinsider.com/bill-gates-cor...,2020-07-30,Business Insider,https://www.businessinsider.com


In [12]:
api_content_current_session.to_csv("../Data/NewsContentTo0729Uncleaned.csv")
api_content.to_csv("../Data/NewsContent.csv")

---

In [ ]:

# import GetOldTweets3 as gt

https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset

In [102]:
# temp1 = temp_cur_session.drop_duplicates(["Title","PublishedTime"])
# api_content_current_session = temp1
# temp_cur_session